In [1]:
import geopandas as gpd
import momepy as mm

In [2]:
mastermap = gpd.read_parquet('../../urbangrammar_samba/OS_MasterMap_Buildings_sample/edinburgh.pq')
openmap = gpd.read_parquet('../../urbangrammar_samba/OS_MasterMap_Buildings_sample/edinburgh_om.pq')
mastermap_tess = gpd.read_parquet('../../urbangrammar_samba/OS_MasterMap_Buildings_sample/edinburgh_mm_tess.pq')
openmap_tess = gpd.read_parquet('../../urbangrammar_samba/OS_MasterMap_Buildings_sample/edinburgh_om_tess.pq')

## Get small sample of morphometric data

In [3]:
mastermap['area'] = mastermap.area
openmap['area'] = openmap.area

In [4]:
openmap_tess['area_t'] = openmap_tess.area
mastermap_tess['area_t'] = mastermap_tess.area

In [5]:
openmap_tess['peri_t'] = openmap_tess.length
mastermap_tess['peri_t'] = mastermap_tess.length

In [8]:
openmap_tess['eri'] = mm.EquivalentRectangularIndex(openmap_tess, 'area_t', 'peri_t').series

In [9]:
mastermap_tess['eri'] = mm.EquivalentRectangularIndex(mastermap_tess, 'area_t', 'peri_t').series

In [10]:
openmap_tess['car'] = mm.AreaRatio(openmap_tess, openmap, 'area_t', 'area', 'uID').series

In [11]:
mastermap_tess['car'] = mm.AreaRatio(mastermap_tess, mastermap, 'area_t', 'area', 'uID').series

In [12]:
mastermap_tess['xID'] = range(len(mastermap_tess))
openmap_tess['xID'] = range(len(openmap_tess))

In [14]:
openmap = openmap.merge(openmap_tess[['uID', 'xID']], on='uID', how='left')
mastermap = mastermap.merge(mastermap_tess[['uID', 'xID']], on='uID', how='left')

In [15]:
from libpysal.weights import Queen

In [16]:
om_w1 = Queen.from_dataframe(openmap_tess, idVariable='xID')

In [17]:
mm_w1 = Queen.from_dataframe(mastermap_tess, idVariable='xID')

In [18]:
om_w3 = mm.sw_high(k=3, weights=om_w1)

In [19]:
mm_w3 = mm.sw_high(k=3, weights=mm_w1)

In [21]:
openmap_tess.columns

Index(['uID', 'geometry', 'eID', 'area_t', 'peri_t', 'eri', 'car', 'xID'], dtype='object')

In [22]:
openmap_tess['area_t_IQR_3steps'] = mm.Range(openmap_tess, 'area_t', om_w3, 'xID', rng=(25, 75)).series

100%|██████████| 74736/74736 [00:46<00:00, 1594.96it/s]


In [23]:
openmap_tess['peri_t_IQR_3steps'] = mm.Range(openmap_tess, 'peri_t', om_w3, 'xID', rng=(25, 75)).series
openmap_tess['eri_IQR_3steps'] = mm.Range(openmap_tess, 'eri', om_w3, 'xID', rng=(25, 75)).series
openmap_tess['car_IQR_3steps'] = mm.Range(openmap_tess, 'car', om_w3, 'xID', rng=(25, 75)).series

100%|██████████| 74736/74736 [00:46<00:00, 1591.84it/s]


In [24]:
mastermap_tess['area_t_IQR_3steps'] = mm.Range(mastermap_tess, 'area_t', mm_w3, 'xID', rng=(25, 75)).series
mastermap_tess['peri_t_IQR_3steps'] = mm.Range(mastermap_tess, 'peri_t', mm_w3, 'xID', rng=(25, 75)).series
mastermap_tess['eri_IQR_3steps'] = mm.Range(mastermap_tess, 'eri', mm_w3, 'xID', rng=(25, 75)).series
mastermap_tess['car_IQR_3steps'] = mm.Range(mastermap_tess, 'car', mm_w3, 'xID', rng=(25, 75)).series

 88%|████████▊ | 66089/74736 [00:41<00:05, 1595.53it/s]


KeyboardInterrupt: 

In [ ]:
openmap_tess = openmap_tess.merge(openmap[['uID', 'area']], on='uID', how='left')
mastermap_tess = mastermap_tess.merge(mastermap[['uID', 'area']], on='uID', how='left')

In [ ]:
openmap_tess['area_IQR_3steps'] = mm.Range(openmap_tess, 'area', om_w3, 'xID', rng=(25, 75)).series

In [ ]:
mastermap_tess['area_IQR_3steps'] = mm.Range(mastermap_tess, 'area', mm_w3, 'xID', rng=(25, 75)).series

## Try k-means